<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [1]:
# import cell
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split
import sklearn.linear_model
import matplotlib.pyplot as plt
import tempfile
from sklearn.utils import shuffle
import os
import datetime
# Our Model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
from keras.layers import Dropout
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorboard.plugins.hparams import api as hp
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam

In [2]:
# Your Code Starts Here - load in data and spilt
data = np.load('quickdraw10.npz')
def load_quickdraw10npz(path):
    X = path['arr_0']
    y = path['arr_1']
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.1)
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = load_quickdraw10npz(path=data)
len(X_train), len(X_test), len(y_train), len(y_test)

(80000, 10000, 80000, 10000)

In [3]:
print(f'shapes{X_train.shape,y_train.shape,X_test.shape,y_test.shape}')

shapes((80000, 784), (80000,), (10000, 784), (10000,))


In [4]:
# Function to create model, required for 
# KerasClassifier all at the same time takes to long
"""def create_model(activation='relu',optimizer='SDG',dropout_rate=0.0, weight_constraint=0,init_mode='uniform',learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=784, kernel_initializer=init_mode, activation=activation, kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    # opt = optimizer(lr=learn_rate, momentum=momentum,name=f'{optimizer}')
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
model = KerasClassifier(build_fn=create_model, epochs=2, batch_size=1, verbose=0)
# define the grid search parameters
#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3,1]
#momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode,activation=activation,optimizer=optimizer,dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))"""

'def create_model(activation=\'relu\',optimizer=\'SDG\',dropout_rate=0.0, weight_constraint=0,init_mode=\'uniform\',learn_rate=0.01, momentum=0):\n    # create model\n    model = Sequential()\n    model.add(Dense(12, input_dim=784, kernel_initializer=init_mode, activation=activation, kernel_constraint=maxnorm(weight_constraint)))\n    model.add(Dropout(dropout_rate))\n    model.add(Dense(1, activation=\'sigmoid\'))\n    # Compile model\n    # opt = optimizer(lr=learn_rate, momentum=momentum,name=f\'{optimizer}\')\n    model.compile(loss=\'binary_crossentropy\', optimizer=optimizer, metrics=[\'accuracy\'])\n    return model\n# fix random seed for reproducibility\nseed = 7\nnp.random.seed(seed)\nmodel = KerasClassifier(build_fn=create_model, epochs=2, batch_size=1, verbose=0)\n# define the grid search parameters\n#learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3,1]\n#momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]\nweight_constraint = [1, 2, 3, 4, 5]\ndropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6,

In [35]:
"""def model(opt, drop, activation, input_shape, output_size, n_layers):
    opt = opt(lr=.001)
    model = Sequential()
    model.add(Dense(50, activation=activation, input_shape=input_shape))
    model.add(Dropout(drop))
    for _ in range(n_layers):
        model.add(Dense(50, activation=activation))
        model.add(Dropout(drop))
    if output_size == 1:
        model.add(Dense(output_size, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', metrics=['mae', 'mse'], optimizer='adam')
    else:
        model.add(Dense(output_size, activation='softmax'))
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
    return model"""

In [28]:
# Function to create model, required for KerasClassifier for batch and epocs
def create_model(lr=.001,opt=SGD):
    opti = opt(lr)
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=784, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation="softmax"))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opti, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
model = KerasClassifier(build_fn=create_model, verbose=1)
# define the grid search parameters
batch_size = [32]
epochs = [32,64]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train,validation_data=(X_test, y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/64
2500/2500 [==============================] - 2s 732us/step - loss: 2.0529 - accuracy: 0.3029 - val_loss: 1.7527 - val_accuracy: 0.3639
Epoch 2/64
2500/2500 [==============================] - 2s 715us/step - loss: 1.6561 - accuracy: 0.4132 - val_loss: 1.5812 - val_accuracy: 0.4455
Epoch 3/64
2500/2500 [==============================] - 2s 730us/step - loss: 1.4970 - accuracy: 0.4718 - val_loss: 1.4632 - val_accuracy: 0.4864
Epoch 4/64
2500/2500 [==============================] - 2s 690us/step - loss: 1.3542 - accuracy: 0.4808 - val_loss: 1.3402 - val_accuracy: 0.4853
Epoch 5/64
2500/2500 [==============================] - 2s 690us/step - loss: 1.2652 - accuracy: 0.5345 - val_loss: 1.2557 - val_accuracy: 0.5517
Epoch 6/64
2500/2500 [==============================] - 2s 677us/step - loss: 1.1649 - accuracy: 0.5900 - val_loss: 1.1689 - val_accuracy: 0.5970
Epoch 7/64
2500/2500 [==============================] - 2s 667us/step - loss: 1.0980 - accuracy: 0.6217 - val_loss: 1.1382 -

In [6]:
# Function to create model, required for KerasClassifier to find best optimizer - Adamax is the best one
def create_model(lr=.001,opt=SGD):
    opti = opt(lr)
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=784, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation="softmax"))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opti, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# model
model = KerasClassifier(build_fn=create_model, epochs=32, batch_size=32, verbose=1)
# define the grid search parameters
opt = [SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam]
param_grid = dict(opt=opt)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train,validation_data=(X_test, y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/32
2500/2500 [==============================] - 2s 774us/step - loss: 3.2490 - accuracy: 0.2869 - val_loss: 1.7042 - val_accuracy: 0.3766
Epoch 2/32
2500/2500 [==============================] - 2s 755us/step - loss: 1.5726 - accuracy: 0.4237 - val_loss: 1.4776 - val_accuracy: 0.4629
Epoch 3/32
2500/2500 [==============================] - 2s 733us/step - loss: 1.4081 - accuracy: 0.4802 - val_loss: 1.3716 - val_accuracy: 0.5023
Epoch 4/32
2500/2500 [==============================] - 2s 735us/step - loss: 1.3102 - accuracy: 0.5218 - val_loss: 1.2937 - val_accuracy: 0.5426
Epoch 5/32
2500/2500 [==============================] - 2s 727us/step - loss: 1.2272 - accuracy: 0.5596 - val_loss: 1.2079 - val_accuracy: 0.5838
Epoch 6/32
2500/2500 [==============================] - 2s 733us/step - loss: 1.1415 - accuracy: 0.6006 - val_loss: 1.1508 - val_accuracy: 0.6125
Epoch 7/32
2500/2500 [==============================] - 2s 718us/step - loss: 1.0709 - accuracy: 0.6355 - val_loss: 1.0821 -

In [5]:
# Function to create model, required for KerasClassifier hp tuning for adamax
def create_model(learn_rate=0.01,beta_1=0.9,beta_2=0.999):
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=784, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation="softmax"))
    # Compile model
    optimizer = Adamax(lr=learn_rate, beta_1=beta_1, beta_2=beta_2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=2, batch_size=512, verbose=1)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
beta_1 = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
beta_2 = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9,0.999]
param_grid = dict(learn_rate=learn_rate, beta_1=beta_1,beta_2=beta_2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train,validation_data=(X_test, y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/2
157/157 [==============================] - 1s 3ms/step - loss: 4.8222 - accuracy: 0.4311 - val_loss: 1.4558 - val_accuracy: 0.5436
Epoch 2/2
157/157 [==============================] - 0s 2ms/step - loss: 1.3135 - accuracy: 0.5780 - val_loss: 1.3571 - val_accuracy: 0.6125
Best: 0.577124 using {'beta_1': 0.2, 'beta_2': 0.2, 'learn_rate': 0.001}
0.555175 (0.017459) with: {'beta_1': 0.0, 'beta_2': 0.0, 'learn_rate': 0.001}
0.513663 (0.043766) with: {'beta_1': 0.0, 'beta_2': 0.0, 'learn_rate': 0.01}
0.101225 (0.002565) with: {'beta_1': 0.0, 'beta_2': 0.0, 'learn_rate': 0.1}
0.100162 (0.001446) with: {'beta_1': 0.0, 'beta_2': 0.0, 'learn_rate': 0.2}
0.100775 (0.002391) with: {'beta_1': 0.0, 'beta_2': 0.0, 'learn_rate': 0.3}
0.553537 (0.019410) with: {'beta_1': 0.0, 'beta_2': 0.2, 'learn_rate': 0.001}
0.474799 (0.043634) with: {'beta_1': 0.0, 'beta_2': 0.2, 'learn_rate': 0.01}
0.116587 (0.022306) with: {'beta_1': 0.0, 'beta_2': 0.2, 'learn_rate': 0.1}
0.101138 (0.002062) with: {'bet

In [6]:
# Function to create model, required for KerasClassifier for Tuning 
# Network Weight Initialization
def create_model2(init_mode='uniform'):
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=784, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(32, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(10, activation="softmax"))
    # Compile model
    optimizer = Adamax(lr=0.001, beta_1=0.2, beta_2=0.2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model2, epochs=16, batch_size=100, verbose=1)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero',
             'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train,validation_data=(X_test, y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/16
800/800 [==============================] - 1s 1ms/step - loss: 2.3072 - accuracy: 0.5164 - val_loss: 1.4895 - val_accuracy: 0.6151
Epoch 2/16
800/800 [==============================] - 1s 965us/step - loss: 1.3522 - accuracy: 0.6288 - val_loss: 1.3952 - val_accuracy: 0.6330
Epoch 3/16
800/800 [==============================] - 1s 969us/step - loss: 1.3326 - accuracy: 0.6471 - val_loss: 1.3301 - val_accuracy: 0.6551
Epoch 4/16
800/800 [==============================] - 1s 968us/step - loss: 1.3305 - accuracy: 0.6611 - val_loss: 1.3862 - val_accuracy: 0.6694
Epoch 5/16
800/800 [==============================] - 1s 1ms/step - loss: 1.2956 - accuracy: 0.6684 - val_loss: 1.3822 - val_accuracy: 0.6556
Epoch 6/16
800/800 [==============================] - 1s 982us/step - loss: 1.2530 - accuracy: 0.7013 - val_loss: 1.3555 - val_accuracy: 0.7022
Epoch 7/16
800/800 [==============================] - 1s 1ms/step - loss: 1.1983 - accuracy: 0.7219 - val_loss: 1.3366 - val_accuracy: 0.718

In [7]:
# Function to create model, required for KerasClassifier
# Tuning the Neuron Activation Function
def create_model(activation='relu'):
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=784, kernel_initializer='glorot_uniform', activation=activation))
    model.add(Dense(32, kernel_initializer='glorot_uniform', activation='relu'))
    model.add(Dense(10, activation="softmax"))
    # Compile model
    optimizer = Adamax(lr=0.001, beta_1=0.2, beta_2=0.2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=16, batch_size=32, verbose=1)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train,validation_data=(X_test, y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/16
2500/2500 [==============================] - 2s 761us/step - loss: 0.8885 - accuracy: 0.7236 - val_loss: 0.7910 - val_accuracy: 0.7606
Epoch 2/16
2500/2500 [==============================] - 2s 753us/step - loss: 0.7623 - accuracy: 0.7716 - val_loss: 0.7759 - val_accuracy: 0.7742
Epoch 3/16
2500/2500 [==============================] - 2s 755us/step - loss: 0.7465 - accuracy: 0.7807 - val_loss: 0.7696 - val_accuracy: 0.7798
Epoch 4/16
2500/2500 [==============================] - 2s 749us/step - loss: 0.7441 - accuracy: 0.7838 - val_loss: 0.7838 - val_accuracy: 0.7867
Epoch 5/16
2500/2500 [==============================] - 2s 737us/step - loss: 0.7448 - accuracy: 0.7894 - val_loss: 0.7869 - val_accuracy: 0.7780
Epoch 6/16
2500/2500 [==============================] - 2s 757us/step - loss: 0.7545 - accuracy: 0.7904 - val_loss: 0.8073 - val_accuracy: 0.7870
Epoch 7/16
2500/2500 [==============================] - 2s 731us/step - loss: 0.7709 - accuracy: 0.7926 - val_loss: 0.7910 -

In [8]:
# Function to create model, required for KerasClassifier
# Tuning Dropout Regularization and wight constrant
def create_model(dropout_rate=0.0, weight_constraint=0):
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=784, kernel_initializer='glorot_uniform', activation='softsign', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, kernel_initializer='glorot_uniform', activation='softmax'))
    # Compile model
    optimizer = Adamax(lr=0.001, beta_1=0.2, beta_2=0.2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=4, batch_size=100, verbose=1)
# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train,validation_data=(X_test, y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/4
800/800 [==============================] - 1s 1ms/step - loss: 1.0350 - accuracy: 0.6904 - val_loss: 0.8370 - val_accuracy: 0.7477
Epoch 2/4
800/800 [==============================] - 1s 1ms/step - loss: 0.7826 - accuracy: 0.7620 - val_loss: 0.7704 - val_accuracy: 0.7696
Epoch 3/4
800/800 [==============================] - 1s 1ms/step - loss: 0.7453 - accuracy: 0.7747 - val_loss: 0.7452 - val_accuracy: 0.7762
Epoch 4/4
800/800 [==============================] - 1s 1ms/step - loss: 0.7275 - accuracy: 0.7809 - val_loss: 0.7423 - val_accuracy: 0.7789
Best: 0.772450 using {'dropout_rate': 0.0, 'weight_constraint': 4}
0.754325 (0.001272) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.766037 (0.001450) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.767687 (0.000685) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.772450 (0.002699) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.772125 (0.000847) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.748837 

In [9]:
# Function to create model, required for KerasClassifier
# Number of Neurons in Hidden Layer and layers
def create_model(neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=784, kernel_initializer='glorot_uniform', activation='softsign', kernel_constraint=maxnorm(4)))
    model.add(Dropout(0))
    model.add(Dense(32, kernel_initializer='glorot_uniform', activation='sigmoid'))
    # Compile model
    optimizer = Adamax(lr=0.001, beta_1=0.2, beta_2=0.2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=4, batch_size=100, verbose=1)
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 32]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train,validation_data=(X_test, y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/4
800/800 [==============================] - 1s 1ms/step - loss: 1.0243 - accuracy: 0.6874 - val_loss: 0.8105 - val_accuracy: 0.7509
Epoch 2/4
800/800 [==============================] - 1s 1ms/step - loss: 0.7752 - accuracy: 0.7640 - val_loss: 0.7579 - val_accuracy: 0.7686
Epoch 3/4
800/800 [==============================] - 1s 1ms/step - loss: 0.7355 - accuracy: 0.7771 - val_loss: 0.7458 - val_accuracy: 0.7667
Epoch 4/4
800/800 [==============================] - 1s 1ms/step - loss: 0.7177 - accuracy: 0.7816 - val_loss: 0.7271 - val_accuracy: 0.7777
Best: 0.769463 using {'neurons': 32}
0.152599 (0.037246) with: {'neurons': 1}
0.584625 (0.022839) with: {'neurons': 5}
0.701788 (0.015427) with: {'neurons': 10}
0.724325 (0.003184) with: {'neurons': 15}
0.748637 (0.004353) with: {'neurons': 20}
0.761213 (0.003397) with: {'neurons': 25}
0.769463 (0.002767) with: {'neurons': 32}


In [12]:
# Function to create model, required for KerasClassifier
# final model
def final_model(neurons=1):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=784, kernel_initializer='glorot_uniform', activation='softsign', kernel_constraint=maxnorm(4)))
    model.add(Dropout(0))
    model.add(Dense(32, kernel_initializer='glorot_uniform', activation='sigmoid'))
    # Compile model
    optimizer = Adamax(lr=0.001, beta_1=0.2, beta_2=0.2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# create model
model = KerasClassifier(build_fn=create_model, epochs=64, batch_size=32, verbose=1)
# define the grid search parameters
neurons = [16, 20, 25, 32, 64]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train,validation_data=(X_test, y_test))
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.8295 - accuracy: 0.7480 - val_loss: 0.7453 - val_accuracy: 0.7761
Epoch 2/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6993 - accuracy: 0.7897 - val_loss: 0.7003 - val_accuracy: 0.7897
Epoch 3/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6759 - accuracy: 0.7973 - val_loss: 0.6849 - val_accuracy: 0.7974
Epoch 4/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6621 - accuracy: 0.8013 - val_loss: 0.6830 - val_accuracy: 0.7917
Epoch 5/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6591 - accuracy: 0.8025 - val_loss: 0.6661 - val_accuracy: 0.7982
Epoch 6/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6586 - accuracy: 0.8030 - val_loss: 0.6752 - val_accuracy: 0.7992
Epoch 7/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6556 - accuracy: 0.8033 - val_loss: 0.6736 - val_accuracy:

In [14]:
def final_model(neurons=64):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=784, kernel_initializer='glorot_uniform', activation='softsign', kernel_constraint=maxnorm(4)))
    model.add(Dropout(0.0))
    model.add(Dense(32, kernel_initializer='glorot_uniform', activation='sigmoid'))
    # Compile model
    optimizer = Adamax(lr=0.001, beta_1=0.2, beta_2=0.2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [15]:
mnist_model_trial1 = final_model(neurons=64)

trial1= mnist_model_trial1.fit(X_train, y_train,
                                # Hyperparameters!
                                epochs=64,
                                batch_size=32,
                                validation_data=(X_test, y_test))

Epoch 1/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.8861 - accuracy: 0.7350 - val_loss: 0.7320 - val_accuracy: 0.7807
Epoch 2/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.7073 - accuracy: 0.7888 - val_loss: 0.6996 - val_accuracy: 0.7899
Epoch 3/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6892 - accuracy: 0.7952 - val_loss: 0.6954 - val_accuracy: 0.7929
Epoch 4/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6772 - accuracy: 0.7992 - val_loss: 0.6887 - val_accuracy: 0.7982
Epoch 5/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6733 - accuracy: 0.8008 - val_loss: 0.6910 - val_accuracy: 0.7957
Epoch 6/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6760 - accuracy: 0.8021 - val_loss: 0.7048 - val_accuracy: 0.7945
Epoch 7/64
2500/2500 [==============================] - 3s 1ms/step - loss: 0.6750 - accuracy: 0.8027 - val_loss: 0.7052 - val_accuracy:

In [16]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### 1. Create Experiment Configuration

In [18]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16,32]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,.01))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adagrad']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
      hparams=[HP_NUM_UNITS, HP_LEARNING_RATE, HP_OPTIMIZER],
      metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')]
  )

### 2. Adapt Model Function with HParams

In [19]:
def train_test_model(hparams):
  
  model = tf.keras.Sequential(
      [tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'),
       tf.keras.layers.Dense(10, activation='softmax')      
  ])

  opt_name = hparams[HP_OPTIMIZER]
  lr = hparams[HP_LEARNING_RATE]

  if opt_name == 'adam':
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
  elif opt_name == 'Adagrad':
    opt = tf.keras.optimizers.Adagrad(learning_rate=lr, initial_accumulator_value=0.1, epsilon=1e-07)
  else:
    raise ValueError(f'Unexpected optimizer: {opt_name}')

  model.compile(
      optimizer=opt,
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )

  model.fit(X_train, y_train, epochs=5)
  _, accuracy = model.evaluate(X_test, y_test)

  # Python convention: if a variable doesn't need a name, give it _
  # ten_ones = [1 for _ in range(10)]

  return accuracy

In [100]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [101]:
session_num = 0

# Basically a grid search
for num_units in HP_NUM_UNITS.domain.values:
  for learning_rate in (HP_LEARNING_RATE.domain.min_value,
                        HP_LEARNING_RATE.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_LEARNING_RATE: learning_rate,
          HP_OPTIMIZER: optimizer
      }

      run_name = f'run-{session_num}'
      print(f'--- Starting trial: {run_name}')
      print({param.name: hparams[param] for param in hparams})
      run('logs/hparams_tuning/' + run_name, hparams)
      session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'learning_rate': 0.001, 'optimizer': 'Adagrad'}
Epoch 1/5
2188/2188 [==============================] - ETA: 0s - loss: 2.2148 - accuracy: 0.06 - ETA: 1s - loss: 1.2655 - accuracy: 0.08 - ETA: 1s - loss: 0.9208 - accuracy: 0.09 - ETA: 1s - loss: 0.7309 - accuracy: 0.09 - ETA: 1s - loss: 0.6096 - accuracy: 0.09 - ETA: 1s - loss: 0.5236 - accuracy: 0.09 - ETA: 1s - loss: 0.4626 - accuracy: 0.09 - ETA: 1s - loss: 0.4151 - accuracy: 0.10 - ETA: 0s - loss: 0.3775 - accuracy: 0.10 - ETA: 0s - loss: 0.3461 - accuracy: 0.10 - ETA: 0s - loss: 0.3200 - accuracy: 0.09 - ETA: 0s - loss: 0.2998 - accuracy: 0.09 - ETA: 0s - loss: 0.2798 - accuracy: 0.09 - ETA: 0s - loss: 0.2641 - accuracy: 0.09 - ETA: 0s - loss: 0.2496 - accuracy: 0.10 - ETA: 0s - loss: 0.2364 - accuracy: 0.10 - ETA: 0s - loss: 0.2248 - accuracy: 0.10 - ETA: 0s - loss: 0.2140 - accuracy: 0.10 - ETA: 0s - loss: 0.2043 - accuracy: 0.09 - ETA: 0s - loss: 0.1959 - accuracy: 0.09 - ETA: 0s - los

### 3. Visualize the Result

In [70]:
%tensorboard --logdir logs/hparams_tuning

Reusing TensorBoard on port 6006 (pid 13036), started 0:21:51 ago. (Use '!kill 13036' to kill it.)

### 4. Hyperparameters with RandomSearchCV (Learn)

In [49]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

"""
This model Tunes:
- Number of Neurons in the Hidden Layer
- Learning Rate in Adam

"""
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=64,
                                        step=32),
                           activation='softsign'))
    model.add(layers.Dense(32, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adamax(
            hp.Choice('learning_rate',
                      values=[0.001])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model
def final_model(neurons=64):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=784, kernel_initializer='glorot_uniform', activation='softsign', kernel_constraint=maxnorm(4)))
    model.add(Dropout(0.0))
    model.add(Dense(32, kernel_initializer='glorot_uniform', activation='sigmoid'))
    # Compile model
    optimizer = Adamax(lr=0.001, beta_1=0.2, beta_2=0.2, epsilon=1e-07)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [51]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='keras-tuner-trial6',
    project_name='helloworld')

In [52]:
tuner.search_space_summary()

In [54]:
tuner.search(X_train, y_train,
             epochs=32,batch_size=32,
             validation_data=(X_test, y_test))

Epoch 1/32
2500/2500 [==============================] - ETA: 0s - loss: 3.4981 - accuracy: 0.03 - ETA: 1s - loss: 3.1542 - accuracy: 0.10 - ETA: 1s - loss: 2.9331 - accuracy: 0.14 - ETA: 1s - loss: 2.7595 - accuracy: 0.16 - ETA: 1s - loss: 2.6407 - accuracy: 0.18 - ETA: 1s - loss: 2.5562 - accuracy: 0.20 - ETA: 1s - loss: 2.4894 - accuracy: 0.22 - ETA: 1s - loss: 2.4353 - accuracy: 0.24 - ETA: 1s - loss: 2.3881 - accuracy: 0.25 - ETA: 1s - loss: 2.3486 - accuracy: 0.26 - ETA: 1s - loss: 2.3047 - accuracy: 0.27 - ETA: 1s - loss: 2.2633 - accuracy: 0.29 - ETA: 1s - loss: 2.2244 - accuracy: 0.30 - ETA: 1s - loss: 2.1882 - accuracy: 0.31 - ETA: 1s - loss: 2.1540 - accuracy: 0.32 - ETA: 1s - loss: 2.1219 - accuracy: 0.33 - ETA: 1s - loss: 2.0930 - accuracy: 0.34 - ETA: 1s - loss: 2.0643 - accuracy: 0.35 - ETA: 1s - loss: 2.0374 - accuracy: 0.36 - ETA: 1s - loss: 2.0128 - accuracy: 0.36 - ETA: 0s - loss: 1.9879 - accuracy: 0.37 - ETA: 0s - loss: 1.9644 - accuracy: 0.38 - ETA: 0s - loss: 1.94

Epoch 1/32
2500/2500 [==============================] - ETA: 0s - loss: 3.4476 - accuracy: 0.03 - ETA: 2s - loss: 2.9214 - accuracy: 0.17 - ETA: 2s - loss: 2.6822 - accuracy: 0.22 - ETA: 2s - loss: 2.5404 - accuracy: 0.24 - ETA: 2s - loss: 2.4432 - accuracy: 0.25 - ETA: 2s - loss: 2.3642 - accuracy: 0.27 - ETA: 1s - loss: 2.2880 - accuracy: 0.29 - ETA: 1s - loss: 2.2230 - accuracy: 0.31 - ETA: 1s - loss: 2.1639 - accuracy: 0.32 - ETA: 1s - loss: 2.1140 - accuracy: 0.33 - ETA: 1s - loss: 2.0701 - accuracy: 0.34 - ETA: 1s - loss: 2.0311 - accuracy: 0.35 - ETA: 1s - loss: 1.9900 - accuracy: 0.36 - ETA: 1s - loss: 1.9535 - accuracy: 0.37 - ETA: 1s - loss: 1.9183 - accuracy: 0.38 - ETA: 1s - loss: 1.8877 - accuracy: 0.39 - ETA: 1s - loss: 1.8590 - accuracy: 0.40 - ETA: 1s - loss: 1.8328 - accuracy: 0.41 - ETA: 1s - loss: 1.8067 - accuracy: 0.42 - ETA: 1s - loss: 1.7824 - accuracy: 0.43 - ETA: 1s - loss: 1.7602 - accuracy: 0.44 - ETA: 1s - loss: 1.7398 - accuracy: 0.44 - ETA: 1s - loss: 1.71

INFO:tensorflow:Oracle triggered exit


In [55]:
tuner.results_summary()

### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 